In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random
from tqdm import tqdm_notebook as tqdm

from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *
from Gauss_fit_functions import extractFIT, extractToPs 

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\os.py
c:\Users\Jason\thesis_project


# Get Gauss-Fit Features

### Use a small subgroup from the dataset to create feature extraction code
##### The profiles chosen must:
##### - Be typical residential users
##### - Have enough data collected for winter/weekdays
##### - Have a gauss fit with a low error

In [2]:
X = genX([1994,2014],drop_0 = True)

dropping all zero rows


In [3]:
# Collect only winter weekday profiles from dataset

X.reset_index(inplace = True)

df = X.copy()

# Extract Season
df['month'] = df.date.dt.month
df['season'] = df['month'].apply(lambda x: 'winter' if x in [6, 7, 8] else 'summer') 
df_winter = df[df['season'] == 'winter'] # Create dataframe with all the winter months, excluding weekends


# Extract Weekdays
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
weekends = ['Sunday', 'Saturday']
df_winter['day_names'] = df_winter.date.dt.day_name()
df_winter['daytype'] = df_winter.day_names.where(~df_winter.day_names.isin(weekdays), 'weekday')
df_winter['daytype'] = df_winter.day_names.where(~df_winter.day_names.isin(weekends), 'weekend')
df_winter.drop(['day_names'], axis = 1, inplace = True)
df_winter_weekend  = df_winter[df_winter['daytype'] == 'weekend'] # Create dataframe with only weekdays
df_winter_weekend.drop(['month', 'season','daytype'], axis = 1, inplace = True)
df_winter_weekend.set_index(['ProfileID','date'], inplace=True)

<ipython-input-3-b58041af8025>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-b58041af8025>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-b58041af8025>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Jason\AppData\Lo

### Get households that pass the stictily residential test

In [4]:
# mean = df_winter_weekend.groupby(["ProfileID"]).mean()

# # Test 1: Energy between 9-5 
# i = np.arange(9,17)
# sum_df = pd.DataFrame(index = mean.index, columns = ["9_to_5", 'total'])
# sum_df = sum_df.fillna(0)
# for x in i:
#     temp = mean.iloc[:,x]
#     sum_df['9_to_5'] = sum_df['9_to_5'] + temp

# sum_df['total'] =  mean.sum(axis = 1)

# sum_df["percentage"] = (sum_df["9_to_5"]/sum_df["total"]) * 100

# def check_9_to_5(df):
#     if df > 50:
#         return 'B'
#     else: 
#         return 'R'

# sum_df['Labels'] = sum_df['percentage'].apply(check_9_to_5)

# # 173 households labeled as Businesses for the 9-to-5 check
# sum_df[sum_df['percentage'] > 50]

,9_to_5,total,percentage,Labels
ProfileID,,,,
66,7.314488,14.168734,51.624145,B
217,29.234047,48.776469,59.934734,B
221,11.493651,22.538165,50.996394,B
232,46.414889,88.394214,52.508967,B
284,22.920155,44.093986,51.980230,B
...,...,...,...,...
12028369,19.029901,37.612317,50.594864,B
12028403,6.608640,8.126843,81.318658,B
12028447,24.028575,45.127989,53.245392,B


In [5]:
# # Tests 2: ToP
# top_df = pd.DataFrame()

# top_df['Time'] = mean.idxmax(axis=1)

# def check_top(df):
#     if int(df) > 9 and int(df) < 17:
#         return 'B'
#     else:
#         return 'R'  

# top_df['Labels'] = top_df['Time'].apply(check_top)

### Extract households that are residential according to test

In [6]:
# new_df = pd.DataFrame()

# new_df['ToP'], new_df['Sum'] = top_df['Labels'], sum_df['Labels']

# data_classes = ['R','B']
# d = dict(zip(data_classes, range(0,2)))

# new_df['ToP'],new_df['Sum'] = new_df['ToP'].map(d, na_action='ignore'),new_df['Sum'].map(d, na_action='ignore')

# new_df['Total'] = new_df.sum(axis = 1)

In [7]:
# residential_ids = new_df[new_df['Total'] == 0].index.values # Contains the profile ids that have typical residential behaviour

# Extract Residential households that have complete data
### Housholds must have:
### - Atleast 3 months of winter weekday data, i.e. +-60 days of data

In [5]:
df_winter_weekend.reset_index(inplace = True)

In [9]:
# # Remove all households with less than three winter months
# winter_months_ids = []

# for i in df_winter_weekdays['ProfileID'].unique():
#     if len(df_winter_weekdays[df_winter_weekdays['ProfileID'] == i]) >= 30:

#         winter_months_ids.append(i) # Contains the profile Ids that have a complete winter months data

# winter_months_ids = np.asarray(winter_months_ids)

In [10]:
# profileIDs = np.intersect1d(winter_months_ids, residential_ids)

In [11]:
# len(profileIDs)

9993

# Filter out all the ids from the above two extractions steps

In [6]:
profileIDs = pd.read_pickle('ProfileIDs.pkl')

In [7]:
cleaned_df = df_winter_weekend[df_winter_weekend['ProfileID'].isin(profileIDs)]
cleaned_df.head()

,ProfileID,date,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,1,1994-06-18,3.691667,0.800000,0.508333,2.183333,0.741667,0.658333,1.991667,0.700000,...,1.450000,1.133333,1.666667,3.916667,9.241667,3.275000,6.325000,2.816667,0.925,2.025000
1,1,1994-06-19,0.808333,0.500000,2.075000,0.691667,0.325000,2.075000,0.691667,0.316667,...,2.125000,2.616667,2.358333,1.141667,7.283333,5.391667,6.441667,9.716667,7.500,0.841667
2,1,1994-06-25,0.683333,1.041667,1.625000,0.566667,1.175000,3.241667,0.425000,0.933333,...,2.075000,2.591667,1.266667,1.333333,1.400000,1.625000,1.700000,1.700000,1.925,2.000000
3,1,1994-06-26,1.925000,1.700000,1.900000,2.000000,2.000000,2.000000,2.000000,2.000000,...,1.275000,2.625000,1.400000,1.333333,1.675000,1.400000,1.675000,1.725000,1.850,2.000000
4,1,1994-07-02,2.250000,0.550000,2.133333,0.200000,0.925000,1.583333,0.200000,2.900000,...,3.433333,5.791667,5.583334,2.608333,1.225000,0.916667,2.975000,2.341667,2.525,0.683333


# Extract gauss_fit features
### Extract the mean amplitudes
### Sigma values
### Mean amplitude deviation
### Mean Daily consumptions
### Deviation in daily consumption
### Time of peaks


In [11]:
cleaned_df[cleaned_df['ProfileID'] == id]

,ProfileID,date,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
253462,12025660,2014-06-01,0.333333,0.333333,0.250000,0.283333,0.183333,0.066667,0.066667,0.083333,...,1.050000,0.916667,0.133333,0.200000,0.116667,3.033333,4.133333,0.266667,0.066667,0.083333
253463,12025660,2014-06-07,0.266667,0.283333,0.266667,0.266667,0.200000,0.200000,1.750000,0.550000,...,0.066667,0.066667,0.166667,0.116667,0.066667,0.116667,1.183333,0.350000,0.266667,0.283333
253464,12025660,2014-06-14,0.366667,0.283333,0.366667,0.283333,0.450000,0.666667,1.150000,1.583333,...,0.433333,0.400000,0.450000,0.316667,0.383333,0.366667,0.416667,0.283333,0.366667,0.300000
253465,12025660,2014-06-15,0.316667,0.366667,0.283333,0.283333,0.266667,0.283333,0.350000,1.133333,...,3.766667,0.900000,0.966667,0.300000,0.200000,0.200000,0.200000,1.366667,0.216667,0.200000
253466,12025660,2014-06-21,0.833333,0.266667,0.266667,0.283333,0.200000,0.266667,2.766667,1.583333,...,0.316667,0.316667,0.066667,0.133333,0.066667,0.083333,0.100000,0.050000,0.050000,0.066667
253467,12025660,2014-06-22,0.033333,0.033333,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,...,1.066667,0.116667,0.050000,0.100000,0.050000,0.100000,0.066667,0.066667,0.066667,0.066667
253468,12025660,2014-06-28,0.416667,0.433333,0.316667,0.416667,0.366667,0.383333,0.400000,0.200000,...,0.000000,0.000000,0.566667,0.000000,0.000000,0.000000,0.000000,0.900000,0.300000,0.400000
253469,12025660,2014-06-29,0.383333,0.266667,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.816667
253470,12025660,2014-07-05,0.166667,0.166667,0.166667,0.133333,0.150000,0.116667,0.133333,0.150000,...,0.500000,0.533333,1.016667,2.566667,4.333333,1.883333,0.266667,0.116667,0.016667,0.083333
253471,12025660,2014-07-06,0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,1.066667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Read in the profiles which were completed

In [8]:
completed_df = pd.read_csv("FitFeatures_dummy_High_season_weekdays.csv")
len(completed_df['ProfileID'].unique())

8878

In [11]:
len(np.intersect1d(profileIDs, completed_df['ProfileID'].unique()))

8878

In [12]:
profileIDs2 = set(profileIDs)^set(completed_df['ProfileID'].unique())
len(profileIDs2)

1115

In [13]:
# # Create Dataframe
# cols = pd.MultiIndex.from_tuples([("ProfileID",''),
#                                 ('H_offset', 'H_offset'),
#                                  ("Morning", "sigma1"), 
#                                   ("Morning", "sigma2"), 
#                                   ("Morning", "mu1"),
#                                   ("Morning", "A1"),
#                                   ("Afternoon", "sigma3"), 
#                                   ("Afternoon", "sigma4"), 
#                                   ("Afternoon", "mu2"),
#                                   ("Afternoon", "A2")])

cols = ['ProfileID','H_offset','sigma1','sigma2','mu1','A1','sigma3','sigma4','mu2','A2']

# Create dummy variables
H_offset = 0
sigma1 = 0
sigma2 = 0 
mu1 = 0
A1 = 0 
sigma3 = 0 
sigma4 = 0 
mu2 = 0
A2 = 0

data=[['DROP_ROW',H_offset,sigma1,sigma2, mu1, A1, sigma3, sigma4, mu2,A2]]

gauss_df = pd.DataFrame(data, columns=cols)
dummy_df = pd.DataFrame(data, columns = cols)
i = 0
for id in tqdm(profileIDs2):
    i = i + 1
    try:
        H_offset,sigma1, sigma2, mu1, A1, sigma3, sigma4, mu2, A2, check = extractFIT(cleaned_df,id)
    except IndexError:
        continue
    if check == False:
        continue

    data=[[id,H_offset,sigma1,sigma2, mu1, A1, sigma3, sigma4, mu2,A2]]
    temp_df = pd.DataFrame(data, columns=cols)
    temp_df.set_index(['ProfileID'])
    gauss_df = gauss_df.append(temp_df)

    if i == 5:
        dummy_df = gauss_df.copy()
        dummy_df = dummy_df.set_index(['ProfileID'])
        # Store Gaussian fit features
        dummy_temp = dummy_df.copy()
        dummy_temp.drop(['DROP_ROW'], axis = 0, inplace = True)
        dummy_temp.to_csv('FitFeatures_dummy2_High_season_weekends.csv')
        i = 0

gauss_df = gauss_df.set_index(['ProfileID'])

# Store Gaussian Fit features
temp = gauss_df.copy()
temp.drop(['DROP_ROW'],axis = 0, inplace = True)
temp.to_csv('FitFeatures_High_season_weekends_2.csv')

<ipython-input-13-badabf341832>:31: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1115 [00:00<?, ?it/s]

In [34]:
# temporary_df = cleaned_df.groupby(["ProfileID"]).mean()

# Combine the two high_season_weekends dataframes

In [17]:
df1 = pd.read_csv('FitFeatures_dummy_High_season_weekends.csv')
df2 = pd.read_csv('FitFeatures_High_season_weekends_2.csv')

In [20]:
combined = pd.concat([df1, df2], ignore_index=True)

In [22]:
combined.to_csv("FitFeatures_High_Season_Weekends.csv")

# Extract the amplitude of every day for each profile

In [35]:
reduced_list = random.choices(profileIDs,k = 10)

In [36]:
t = cleaned_df[cleaned_df['ProfileID'] == 5]
t_series = t.iloc[0]
t_series.drop(labels = ['date'], inplace = True)
t_frame = t_series.to_frame()
t_frame = t_frame.T
t_frame.reset_index(inplace = True)
t_frame.drop(['index'], axis = 1, inplace = True)
t_frame.set_index(['ProfileID'], inplace = True)
t_frame
# df = df.drop(labels = ['ProfileID', 'date'])

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4443: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
ProfileID,,,,,,,,,,,,,,,,,,,,,
5,2.97775,1.069333,1.363583,2.370667,1.098,2.6325,7.26575,8.111167,11.583917,9.582167,...,9.444333,17.527,13.849,11.341917,9.104583,15.9335,17.159333,6.229917,4.240667,4.9265


In [37]:
# Create cols
cols = ['ProfileID','A1','A2','mu1','mu2']

# Create dummy variables
mu1 = 0
A1 = 0 
mu2 = 0
A2 = 0

data=[['DROP_ROW',A1,A2,mu1,mu2]]

amplitudes_df = pd.DataFrame(data, columns=cols)

for id in profileIDs:
    for index in cleaned_df[cleaned_df['ProfileID'] == id].index:
        A1, A2, mu1, mu2, check = extractToPs(cleaned_df[cleaned_df['ProfileID'] == id].loc[index])
        if check == False:
            continue
        
        data=[[id,A1,A2,mu1,mu2]]
        temp_df = pd.DataFrame(data, columns=cols)
        # temp_df.set_index(['ProfileID'])
        amplitudes_df = amplitudes_df.append(temp_df)
    
amplitudes_df = amplitudes_df.set_index(['ProfileID'])
temp = amplitudes_df.copy()
temp.drop(['DROP_ROW'],axis = 0, inplace = True)
    

# Calculate the amplitude deviations

In [38]:
temp.head()

,A1,A2,mu1,mu2
ProfileID,,,,
1,4.808333,10.433333,6,21
1,4.066667,12.908333,6,21
1,8.750000,13.125000,10,20
1,4.216667,10.466667,6,20
1,3.783333,5.400000,6,14


In [39]:
def standard_deviation(my_list):
    #calculate population standard deviation of list 
    return (sum((x-(sum(my_list) / len(my_list)))**2 for x in my_list) / len(my_list))**0.5

In [40]:
daily_consumption = pd.DataFrame()

daily_consumption['Daily_Consumption'] = cleaned_df.set_index(["ProfileID"]).sum(axis = 1)

std_deviation_df = pd.DataFrame(index = temp.index.unique())

for id in temp.index.unique():
    std_deviation_df.loc[id,'A1_std'] = standard_deviation(temp.loc[id]['A1'])
    std_deviation_df.loc[id,'A2_std'] = standard_deviation(temp.loc[id]['A2'])
    std_deviation_df.loc[id,'mu1_std'] = standard_deviation(temp.loc[id]['mu1'])
    std_deviation_df.loc[id,'mu2_std'] = standard_deviation(temp.loc[id]['mu2'])
    std_deviation_df.loc[id,'DC_std'] = standard_deviation(daily_consumption.loc[id]['Daily_Consumption'])


In [41]:
std_deviation_df

,A1_std,A2_std,mu1_std,mu2_std,DC_std
ProfileID,,,,,
1,1.524691,4.144478,1.572146,2.016839,15.503554
5,3.734773,4.208459,1.066960,2.274852,25.233687
6,2.612709,4.590361,1.149413,1.614234,18.749896
10,1.586904,1.996307,1.533401,1.876822,13.320836
11,1.880715,2.051301,1.431970,1.579903,14.821876
...,...,...,...,...,...
12028746,3.496038,4.944541,2.625675,2.319962,24.696855
12028750,5.085144,2.385060,2.082483,2.367873,20.613692
12028754,0.912373,3.094797,1.625833,1.727152,6.670844


# Combine the features extracted for the synthetic profiles

In [42]:
# std_deviation_df
# temp.columns = temp.columns.droplevel(0)

combined_df = pd.merge(temp,std_deviation_df, left_index = True, right_index = True)
combined_df.to_csv('FitFeaturescombined_ALL.csv')

In [43]:
combined_df

,A1,A2,mu1,mu2,A1_std,A2_std,mu1_std,mu2_std,DC_std
ProfileID,,,,,,,,,
1,4.808333,10.433333,6,21,1.524691,4.144478,1.572146,2.016839,15.503554
1,4.066667,12.908333,6,21,1.524691,4.144478,1.572146,2.016839,15.503554
1,8.750000,13.125000,10,20,1.524691,4.144478,1.572146,2.016839,15.503554
1,4.216667,10.466667,6,20,1.524691,4.144478,1.572146,2.016839,15.503554
1,3.783333,5.400000,6,14,1.524691,4.144478,1.572146,2.016839,15.503554
...,...,...,...,...,...,...,...,...,...
12029049,1.683333,1.533333,10,17,0.838400,1.103542,2.409524,1.253057,4.146102
12029049,0.450000,1.116667,5,17,0.838400,1.103542,2.409524,1.253057,4.146102
12029049,1.466667,1.450000,6,18,0.838400,1.103542,2.409524,1.253057,4.146102


In [44]:
# def extract_all_amplitudes(df,id):

#     A1, A2, mu1, mu2 = extractToPs(df, id)

#     data=[[id, A1, A2, mu1, mu2]]

#     temp_df = pd.DataFrame(data, columns=cols)
#     temp_df.set_index(['ProfileID'])
#     gauss_df = gauss_df.append(temp_df)

In [45]:
# for id in reduced_list:
#     for row in cleaned_df.loc[id]:
#         print(row)


### Winter Weekdays

In [46]:
# path =r'C:\Users\Jason\thesis_project\data\features\FitFeatures\Winter_Weekdays'
# list_files = []
# for root, dirs, files in os.walk(path):
# 	for file in files:
# 		list_files.append(os.path.join(root,file))

# dfs = []
# res = ''
# for name in list_files:
#     temp_df = pd.read_csv(name,header=[0, 1], index_col = 0)
#     dfs.append(temp_df)

# res = pd.concat(dfs, ignore_index=False)  # concatenate list of dataframes

# res

# Get the amplitude deviation

### Winter-weekdays

In [47]:
# X.reset_index(inplace = True)

# df = X.copy()

# # Extract Season
# df['month'] = df.date.dt.month
# df['season'] = df['month'].apply(lambda x: 'winter' if x in [6, 7, 8] else 'summer') 
# df_winter = df[df['season'] == 'winter'] # Create dataframe with all the winter months, excluding weekends


# # Extract Weekdays
# weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
# df_winter['day_names'] = df_winter.date.dt.day_name()
# df_winter['daytype'] = df_winter.day_names.where(~df_winter.day_names.isin(weekdays), 'weekday')
# df_winter.drop(['day_names'], axis = 1, inplace = True)
# df_winter_weekdays  = df_winter[df_winter['daytype'] == 'weekday'] # Create dataframe with only weekdays
# df_winter_weekdays.drop(['month', 'season','daytype'], axis = 1, inplace = True)
# df_winter_weekdays.set_index(['ProfileID','date'], inplace=True)

# # df_winter_weekdays.to_csv("Winter_Weekdays_Dataframe.csv")


# Extract the amplitude devition

In [48]:
# # Get the profile IDs
# ids = res.index.values

In [49]:
# df_winter_weekdays.reset_index(["date"],inplace = True)

In [50]:
# df_winter_weekdays = df_winter_weekdays[df_winter_weekdays.index.isin(ids)]

In [51]:
# df_winter_weekdays.drop(['date'],axis = 1,inplace = True)

In [52]:
# df_winter_weekdays.reset_index(inplace = True)

In [53]:
# df_mean = df_winter_weekdays.groupby(['ProfileID']).mean()